In [ ]:
import os
import glob
import numpy as np
import h5py
import xesmf as xe
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import Geod
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.colors as mcolors

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", module="xarray.coding.times")
warnings.filterwarnings("ignore", module="xarray.coding.cftime_offsets")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="xarray")
warnings.filterwarnings("ignore", module="xarray")

import xarray as xr

# CONFIGURACIÓN
bases_in = ["E:/TFG", "D:/TFG"]
escenario = "SSP126"   # o "SSP245"
base_out = f"C:/Users/jaime/Desktop/Universidad/TFG/Resultados/{escenario}/temperature trends"
modelos = ["CESM2-WACCM", "GFDL-CM4", "IPSL-CM6A-LR", "MPI-ESM1-2-LR", "NorESM2-LM","ACCESS-CM2",
           "HadGEM3-GC31-LL","EC-Earth3-Veg-LR","MIROC6","TaiESM1"]  # pon los nombres exactos de tus carpetas
# ,"EC-Earth3-Veg-LR","MIROC6","TaiESM1","CESM2-WACCM", "GFDL-CM4", "IPSL-CM6A-LR", "MPI-ESM1-2-LR", 
# "NorESM2-LM","EC-Earth3-Veg-LR","MIROC6","TaiESM1","ACCESS-CM2"
# FUNCIÓN PARA ABRIR Y PROCESAR

def abrir_y_procesar(archivos, varname):
    datasets_proc = []

    # Definir rejilla destino (2.5° x 2°)
    ds_out = {
        'lon': np.arange(280, 354, 2.5),
        'lat': np.arange(35, 70, 2.0),
    }

    regridder = None  # inicializamos fuera del bucle

    for f in archivos:
        # CAMBIO 1: abrir evitando decode_times y con fallback de backend
        try:
            ds = xr.open_dataset(f, decode_times=False)
        except Exception:
            ds = xr.open_dataset(f, engine="h5netcdf", decode_times=False)

        if 'time' in ds.variables:
            ds = xr.decode_cf(ds, use_cftime=True)

        # CAMBIO 2: detectar archivos corruptos y omitirlos
        try:
            _ = ds[list(ds.data_vars)[0]].isel({list(ds[list(ds.data_vars)[0]].dims)[0]: 0}).load()
        except Exception as e:
            print(f" Archivo problemático, se omite: {f} -> {e}")
            continue

        # Ajustar longitudes a 0–360 por seguridad
        if 'lon' in ds.coords and (ds.lon < 0).any():
            ds = ds.assign_coords(lon=((ds.lon + 360) % 360))

        # Selección estacional DJF
        ds_djf = ds.sel(time=ds['time'].dt.month.isin([12, 1, 2]))

        # Subconjunto espacial y vertical
        ds_sub = ds_djf.sel(
            plev=slice(100000, 10000),  # niveles (Pa)
            lat=slice(34, 71),
            lon=slice(279, 360)
        )

        # Crear el regridder solo en la primera iteración
        # CAMBIAR FALSE A TRUE DESPUÉS DE LA PRIMERA ITERACIÓN DEL CÓDIGO
        if regridder is None:
            regridder = xe.Regridder(ds_sub, ds_out, method='bilinear', reuse_weights=False)

        # Aplicar el remapeo
        ds_remap = regridder(ds_sub)

        datasets_proc.append(ds_remap)

    # Concatenar en tiempo y filtrar años
    ds_concat = xr.concat(datasets_proc, dim="time")
    return ds_concat.where(ds_concat["time"].dt.year >= 2030, drop=True)[varname]

# Función para que diciembre del año anterior
# sea parte del invierno de enero y febrero del año siguiente

def asignar_invierno(ds):
    """Añade coordenada winter_year para DJF (dic -> año+1)."""
    meses = ds['time'].dt.month
    años   = ds['time'].dt.year
    winter_year = xr.where(meses == 12, años + 1, años)
    ds = ds.assign_coords(winter_year=("time", winter_year.data))
    return ds

# EJEMPLO RUTA TRANSATLÁNTICA
lon_DUB, lat_DUB = -6.2621, 53.4287   # Dublín
lon_JFK, lat_JFK = -73.7781, 40.6413  # Nueva York JFK
geod = Geod(ellps="WGS84")

In [ ]:
# BUCLE POR MODELOS - MAPAS
tendencias_modelos_barras = []
tendencias_modelos_mapas = []

# CALCULAR RANGO GLOBAL X PARA TODAS LAS BARRAS
limites_globales = []

for modelo in modelos:
    print(f"Verificando rango de {modelo}...")

    archivos_ta = []
    
    for base_in in bases_in:
        ruta_ta = os.path.join(base_in, modelo, escenario, "ta")
        archivos_ta = sorted(glob.glob(os.path.join(ruta_ta, "*.nc")))
        
        if archivos_ta:
            break  # si encuentra archivos, sale del bucle y los usa
    
    if not archivos_ta:
        print(f"{modelo}: no se encontraron todos los archivos necesarios en ninguna base.")
        continue

    ta = abrir_y_procesar(archivos_ta, "ta")
    ta_with_year = asignar_invierno(ta)
    ta_djf_yearly = ta_with_year.groupby("winter_year").mean("time")
    ta_trend = ta_djf_yearly.polyfit(dim="winter_year", deg=1)
    ta_slope_decadal = ta_trend["polyfit_coefficients"].sel(degree=1) * 10

    ta_slope_profile_decadal = ta_slope_decadal.mean(dim=("lat", "lon"))
    
    # Guardar extremos del perfil
    limites_globales.append(float(ta_slope_profile_decadal.min()))
    limites_globales.append(float(ta_slope_profile_decadal.max()))

# Calcular límites globales simétricos
limite_max = max(abs(min(limites_globales)), abs(max(limites_globales)))
xlim_global = (-limite_max, limite_max)
print(f"\nRango global para eje X: {xlim_global}")

# Escala de colores global (para mapas)
lim_global = limite_max  # usar el mismo rango global que ya calculaste
norm = mcolors.TwoSlopeNorm(vmin=-lim_global, vcenter=0, vmax=lim_global)

for modelo in modelos:
    print(f"Procesando {modelo}...")
    
    archivos_ta = []  # inicializa vacía para buscar en ambas rutas
    
    for base_in in bases_in:
        ruta_ta = os.path.join(base_in, modelo, escenario, "ta")
        archivos_ta = sorted(glob.glob(os.path.join(ruta_ta, "*.nc")))
        if archivos_ta:
            print(f"   Archivos encontrados en: {ruta_ta}") 
            break

    if not archivos_ta:
        print(f" No se encontraron archivos para {modelo}, se omite.")
        continue

    # Abrir datos
    ta = abrir_y_procesar(archivos_ta, "ta")

    ######
    # Tendencia para velocidad zonal (oeste a este)
    # Calcular tendencia en cada gridpoint
    ta_with_year = asignar_invierno(ta)
    ta_djf_yearly = ta_with_year.groupby("winter_year").mean("time")
    
    ta_trend = ta_djf_yearly.polyfit(dim="winter_year", deg=1)
    ta_slope = ta_trend["polyfit_coefficients"].sel(degree=1)  # K por año

    # Aplicar en cada punto del campo usando xarray
    slope_linreg, pval_linreg = xr.apply_ufunc(
        linreg_pval,
        ta_djf_yearly["winter_year"],
        ta_djf_yearly,
        input_core_dims=[["winter_year"], ["winter_year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
    )

    # Convertir a K/década
    ta_slope_decadal = slope_linreg * 10

    # Máscara booleana de significancia (p<0.05)
    significativos = pval_linreg < 0.05
    
    # Convertir a K por década (opcional)
    ta_slope_decadal = ta_slope * 10

    # Promediar espacialmente para tener perfil vertical
    ta_slope_profile_decadal = ta_slope_decadal.mean(dim=("lat", "lon"))

     # Guardamos el campo para la media multi-modelo
    ta_slope_profile_decadal = ta_slope_profile_decadal.assign_coords(modelo=modelo)
    tendencias_modelos_barras.append(ta_slope_profile_decadal)
    
    # Graficar barras por nivel de presión
    ta_plevs = ta_slope_profile_decadal["plev"].values / 100  # hPa
    ta_vals = ta_slope_profile_decadal.values
    
    # Colores según signo de la tendencia
    colors = ["red" if val > 0 else "blue" for val in ta_vals]
    
    fig, ax = plt.subplots(figsize=(3, 4))
    
    ax.barh(ta_plevs, ta_vals, color=colors, height=40)  # grosor fijo para todas las barras
    ax.axvline(0, color="k", linewidth=1)                # línea vertical en x=0
    
    ax.invert_yaxis()                                    # presión mayor abajo
    ax.set_yticks(ta_plevs)
    ax.set_yticklabels(ta_plevs.astype(int))
    
    ax.set_xlabel("K por década", fontsize=8.5)
    ax.set_ylabel("Nivel de presión (hPa)", fontsize=8.5)
    ax.set_title(f"Tendencia de variación de temperatura por década DJF (2030–2100)\n{modelo} {escenario}", fontsize=9)
    
    import matplotlib.ticker as mticker
     # rango del eje x
    margen = 0.05 * limite_max
    xlim_global = (-limite_max - margen, limite_max + margen)
    ax.set_xlim(xlim_global)
                            
    ax.xaxis.set_major_locator(mticker.MultipleLocator(0.1))
    ax.tick_params(axis="both", which="major", labelsize=8.5)
    
    # Crear la carpeta principal si no existe
    os.makedirs(base_out, exist_ok=True)
    
    # Guardar directamente en la carpeta principal
    out_file = os.path.join(base_out, f"{modelo}_barras_ta_tendencia.png")
    fig.savefig(out_file, bbox_inches="tight", dpi=300)
    plt.close(fig)
    
    print(f"   Guardado en {out_file}")
    ######

    # FIGURA INDIVIDUAL
    fig = plt.figure(figsize=(7, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    ta_map_250 = ta_slope_decadal.sel(plev=25000, method='nearest')  # campo a 250 hPa
    
    # Graficar tendencia con escala global y centrada en 0
    im = ta_map_250.plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap="RdBu_r",
        norm=norm,  # usa la escala global definida antes del bucle
        cbar_kwargs={
            "label": "Tendencia (K por década)",
            "orientation": "horizontal",
            "pad": 0.1,
            "shrink": 0.7
        }
    )
    
    im.colorbar.ax.tick_params(labelsize=8)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_extent([285, 354, 36, 68], crs=ccrs.PlateCarree())
    
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {"size": 8}
    gl.ylabel_style = {"size": 8}
    
    plt.title(f"Tendencia de T a 250 hPa (DJF, 2030–2100)\n{modelo} {escenario}", fontsize=8.5)
    
    # Crear carpeta si no existe y guardar
    os.makedirs(base_out, exist_ok=True)
    out_file = os.path.join(base_out, f"{modelo}_mapa_ta_tendencia.png")
    plt.savefig(out_file, bbox_inches="tight", dpi=300)
    plt.close(fig)
    
    print(f"   Guardado en {out_file}")

        ### Mapa de significancia (p<0.05)
    fig = plt.figure(figsize=(7, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Muestra solo tendencias significativas
    ta_map_250_sig = ta_slope_decadal.sel(plev=25000, method='nearest').where(
        significativos.sel(plev=25000, method='nearest')
    )

    im = ta_map_250_sig.plot(
        ax=ax, transform=ccrs.PlateCarree(),
        cmap="RdBu_r", norm=norm,
        cbar_kwargs={
            "label": "Tendencia significativa (K por década, p<0.05)",
            "orientation": "horizontal", "pad": 0.1, "shrink": 0.7
        }
    )

    # Añadir puntos grises donde NO es significativa
    ax.contourf(
        ta_map_250_sig["lon"],
        ta_map_250_sig["lat"],
        ~significativos.sel(plev=25000, method='nearest'),
        levels=[0.5, 1.5],
        colors="none", hatches=["..."],
        transform=ccrs.PlateCarree(),
    )

    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_extent([285, 354, 36, 68], crs=ccrs.PlateCarree())
    plt.title(f"Tendencias significativas (t-test, p<0.05)\n{modelo} {escenario}", fontsize=8.5)
    out_file_sig = os.path.join(base_out, f"{modelo}_mapa_ta_significativo.png")
    plt.savefig(out_file_sig, bbox_inches="tight", dpi=300)
    plt.close(fig)
    print(f"   Guardado mapa de significancia en {out_file_sig}")
    
    # Añadir el campo a la lista multimodelo
    ta_map_250 = ta_map_250.assign_coords(modelo=modelo)
    tendencias_modelos_mapas.append(ta_map_250)

    # Guardar mapa de significancia (booleano o p-valor)
    significativos_250 = significativos.sel(plev=25000, method="nearest")
    significativos_250 = significativos_250.assign_coords(modelo=modelo)
    significancia_modelos_mapas.append(significativos_250)


In [ ]:
# MEDIA MULTI-MODELO - MAPAS
if tendencias_modelos_mapas:
    print("\nCalculando media multi-modelo...")

    tendencias_ds = xr.concat(tendencias_modelos_mapas, dim="modelo")
    tendencia_media = tendencias_ds.mean("modelo")

    # Combinar máscaras de significancia de todos los modelos
    signif_ds = xr.concat(significancia_modelos_mapas, dim="modelo")

    # Criterio de robustez tipo IPCC
    
    # Más del 50% de los modelos con tendencia significativa (p<0.05)
    signif_fraction = signif_ds.sum("modelo") / signif_ds.sizes["modelo"]
    cond_signif = signif_fraction >= 0.5
    
    # Al menos 80% de los modelos con el mismo signo de cambio
    signo_ds = xr.where(tendencias_ds > 0, 1, -1)
    signo_medio = signo_ds.mean("modelo")
    cond_signo = abs(signo_medio) >= 0.8  # 80% o más de acuerdo
    
    # Combinación final
    robusto_multimodel = cond_signif & cond_signo

    # FIGURA MULTI-MODELO
    fig = plt.figure(figsize=(7, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # definir normalización simétrica alrededor de 0
    lim = max(abs(float(tendencia_media.min())), abs(float(tendencia_media.max())))
    norm = mcolors.TwoSlopeNorm(vmin=-lim, vcenter=0, vmax=lim)
    
    im = tendencia_media.plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap="RdBu_r",
        norm=norm,
        cbar_kwargs={
            "label": "Tendencia K por década DJF multimodelo a 250hPa",
            "orientation": "horizontal",
            "pad": 0.1,
            "shrink": 0.7
        }
    )


    im.colorbar.ax.tick_params(labelsize=8)
    im.colorbar.set_label("Tendencia K por década DJF multimodelo a 250hPa", size=8.5)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_extent([285, 354, 36, 68], crs=ccrs.PlateCarree())
    
    # Añadir la ruta DUB–JFK
    puntos = geod.npts(lon_DUB, lat_DUB, lon_JFK, lat_JFK, 100)
    lons = [lon_DUB] + [p[0] for p in puntos] + [lon_JFK]
    lats = [lat_DUB] + [p[1] for p in puntos] + [lat_JFK]
    ax.plot(lons, lats, transform=ccrs.Geodetic(),
            color="black", linewidth=0.8, linestyle="-")

    gl = ax.gridlines(draw_labels=True, linewidth=0.4, color="gray", alpha=0.6, linestyle="--")
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {"size": 7}
    gl.ylabel_style = {"size": 7}
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    plt.title(f"Media multi-modelo de tendencias de temperatura a 250 hPa\n{escenario} DJF (2030–2100)", fontsize=8.5)

    out_file = os.path.join(base_out, "media_multimodelo_temperatura_tendencia.png")
    plt.savefig(out_file, bbox_inches="tight", dpi=300)
    plt.close(fig)

    # FIGURA: MAPA MULTIMODELO DE SIGNIFICANCIA
    fig = plt.figure(figsize=(7, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Muestra solo las tendencias significativas según el criterio multimodelo
    tendencia_sig = tendencia_media.where(robusto_multimodel)

    im = tendencia_sig.plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap="RdBu_r",
        norm=norm,
        cbar_kwargs={
            "label": "Tendencia significativa multimodelo (K por década)",
            "orientation": "horizontal",
            "pad": 0.1,
            "shrink": 0.7
        }
    )

    # Marcar zonas NO significativas con puntos grises
    ax.contourf(
        tendencia_media["lon"],
        tendencia_media["lat"],
        ~robusto_multimodel,
        levels=[0.5, 1.5],
        colors="none",
        hatches=["..."],
        transform=ccrs.PlateCarree(),
    )

    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_extent([285, 354, 36, 68], crs=ccrs.PlateCarree())

    plt.title(f"Tendencias significativas multimodelo (t-test p<0.05)\n{escenario} DJF (2030–2100)", fontsize=8.5)
    out_file_sig = os.path.join(base_out, "media_multimodelo_tendencia_significativa.png")
    plt.savefig(out_file_sig, bbox_inches="tight", dpi=300)
    plt.close(fig)
    print(f"Figura multimodelo de significancia guardada en {out_file_sig}")


    print(f"Figura multi-modelo guardada en {out_file}")

In [ ]:
# MEDIA MULTI-MODELO - BARRAS
if tendencias_modelos_barras:
    print("\nCalculando media multi-modelo...")

    tendencias_ds = xr.concat(tendencias_modelos_barras, dim="modelo")
    tendencia_media = tendencias_ds.mean("modelo")
    tendencia_std   = tendencias_ds.std("modelo")

    # FIGURA MULTI-MODELO
    ta_plevs = tendencia_media["plev"].values / 100  # hPa
    ta_vals  = tendencia_media.values
    ta_err   = tendencia_std.values

    colors = ["red" if val > 0 else "blue" for val in ta_vals]

    fig, ax = plt.subplots(figsize=(3.5, 5))

    # Altura de barras proporcional al espaciado entre niveles
    height = 40

    # Barras principales
    ax.barh(ta_plevs, ta_vals, color=colors, height=height, xerr=ta_err,
            ecolor="k", capsize=2, alpha=0.8)

    ax.axvline(0, color="k", linewidth=1)
    ax.invert_yaxis()
    ax.set_yticks(ta_plevs)
    ax.set_yticklabels(ta_plevs.astype(int))

    ax.set_xlabel("K por década", fontsize=9)
    ax.set_ylabel("Nivel de presión (hPa)", fontsize=9)
    ax.set_title(f"Tendencia de temperatura en DJF (2030–2100)\nMedia multimodelo \n{escenario}", fontsize=10)

    import matplotlib.ticker as mticker
    ax.xaxis.set_major_locator(mticker.MultipleLocator(0.1))
    ax.tick_params(axis="both", which="major", labelsize=8.5)

    # Guardar
    out_file = os.path.join(base_out, "media_multimodelo_barras_temperatura_tendencia.png")
    fig.savefig(out_file, bbox_inches="tight", dpi=300)
    plt.close(fig)

    print(f"Figura multi-modelo guardada en {out_file}")

# MEDIA Y TENDENCIA MULTIMODELO DE TEMPERATURA (K)

print("\n Calculando y guardando medias y tendencias multimodelo de temperatura...")

ta_anuales = []
ta_tendencias = []

for modelo in modelos:
    try:
        # Buscar archivos para este modelo
        archivos_ta = []
        for base_in in bases_in:
            ruta_ta = os.path.join(base_in, modelo, escenario, "ta")
            archivos_ta = sorted(glob.glob(os.path.join(ruta_ta, "*.nc")))
            if archivos_ta:
                break

        if not archivos_ta:
            print(f" No se encontraron archivos para {modelo}, se omite.")
            continue

        # Abrir datos
        ta = abrir_y_procesar(archivos_ta, "ta")

        # Eliminar coord 'modelo' si ya existe (compatible con DataArray)
        if "modelo" in ta.coords:
            ta = ta.drop_vars("modelo", errors="ignore")

        # Agrupar por inviernos DJF
        ta_with_year = asignar_invierno(ta)
        ta_djf_yearly = ta_with_year.groupby("winter_year").mean("time")

        # Guardar climatología (promedio DJF anual)
        ta_mean = ta_djf_yearly.mean("winter_year")
        if "modelo" not in ta_mean.dims:
            ta_mean = ta_mean.expand_dims(modelo=[modelo])
        else:
            ta_mean = ta_mean.assign_coords(modelo=[modelo])
        ta_anuales.append(ta_mean)
        print(f"Media DJF calculada para {modelo}")

        # Calcular tendencia (K/década)
        ta_trend = ta_djf_yearly.polyfit(dim="winter_year", deg=1)
        ta_slope = ta_trend["polyfit_coefficients"].sel(degree=1) * 10  # K/década

        if "modelo" not in ta_slope.dims:
            ta_slope = ta_slope.expand_dims(modelo=[modelo])
        else:
            ta_slope = ta_slope.assign_coords(modelo=[modelo])
        ta_tendencias.append(ta_slope)
        print(f"Tendencia calculada para {modelo}")

    except Exception as e:
        print(f"Error procesando {modelo}: {e}")

# MEDIA MULTIMODELO (CLIMATOLOGÍA)
if len(ta_anuales) == 0:
    raise ValueError("No se pudo calcular ninguna media multimodelo de temperatura.")
else:
    ta_concat = xr.concat(ta_anuales, dim="modelo")
    ta_mean_mm = ta_concat.mean("modelo", skipna=True)

    ta_mean_path = os.path.join(base_out, f"TA_media_multimodelo_{escenario}.nc")
    ta_mean_mm.to_netcdf(ta_mean_path)
    print(f"Guardado: {ta_mean_path}")

# TENDENCIA MULTIMODELO (K/década)
if len(ta_tendencias) == 0:
    raise ValueError("No se pudo calcular ninguna tendencia multimodelo de temperatura.")
else:
    ta_tend_concat = xr.concat(ta_tendencias, dim="modelo")
    ta_tend_mm = ta_tend_concat.mean("modelo", skipna=True)

    ta_tend_path = os.path.join(base_out, f"TA_tendencia_multimodelo_{escenario}.nc")
    ta_tend_mm.to_netcdf(ta_tend_path)
    print(f" Guardado: {ta_tend_path}")

print("\n Cálculo y guardado multimodelo de temperatura completado.")